In [66]:
!pip install scikit-surprise

  Running setup.py install for scikit-surprise: started
    Running setup.py install for scikit-surprise: finished with status 'done'


In [2]:
import numpy as np
import pandas as pd
from sklearn import cross_validation as cv
import random
from surprise import NMF
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import BaselineOnly
from surprise import NormalPredictor
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

c:\softwaress\tools\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
path = "C:/Users/wkl31415926/OneDrive/Projects/CPRS/process/"

In [6]:
def matrix_to_col_dataSet(matrix, reader):
    ret = []
    for idx_r,r in enumerate(matrix):
        for idx_c,c in enumerate(r):
            if not np.isnan(c):
                ret.append([idx_r,idx_c,c])
    ret = pd.DataFrame(ret)
    ret.columns = ['userID', 'itemID', 'rating']
    return Dataset.load_from_df(ret[['userID', 'itemID', 'rating']], reader)

In [13]:
# 读取csv文件
df = pd.read_csv(path + "CF.csv")
df.head()
data = df.drop(["Unnamed: 0"], axis=1, inplace=False).as_matrix()
data = np.log(data)
# 转换格式为 surprise.dataset.DatasetAutoFolds
reader = Reader(rating_scale=(0, 6))
data = matrix_to_col_dataSet(data, reader);

# Baseline

* **NormalPredictor**:假设数据符合高斯分布，根据分布瞎random

In [14]:
algo = NormalPredictor()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4159  1.4097  1.4070  1.4080  1.4065  1.4094  0.0034  
MAE (testset)     1.1385  1.1337  1.1337  1.1313  1.1300  1.1334  0.0029  
Fit time          0.48    0.56    0.56    0.55    0.56    0.54    0.03    
Test time         0.65    0.62    0.62    0.62    0.64    0.63    0.01    


{'fit_time': (0.48394322395324707,
  0.5640027523040771,
  0.561046838760376,
  0.5479536056518555,
  0.5554516315460205),
 'test_mae': array([1.13848226, 1.13374444, 1.13370534, 1.13126946, 1.13004275]),
 'test_rmse': array([1.41590265, 1.40970118, 1.40703903, 1.40801481, 1.40649821]),
 'test_time': (0.6473379135131836,
  0.61617112159729,
  0.6202168464660645,
  0.6176469326019287,
  0.6412687301635742)}

In [15]:
algo = BaselineOnly()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7723  0.7709  0.7716  0.7686  0.7735  0.7713  0.0016  
MAE (testset)     0.6117  0.6101  0.6132  0.6101  0.6148  0.6120  0.0018  
Fit time          1.00    1.47    1.30    1.18    1.07    1.20    0.17    
Test time         0.58    3.94    0.95    0.96    0.58    1.40    1.28    


{'fit_time': (0.9960529804229736,
  1.4710934162139893,
  1.3045239448547363,
  1.1753170490264893,
  1.0745809078216553),
 'test_mae': array([0.61174225, 0.61014851, 0.61316847, 0.61009224, 0.6148017 ]),
 'test_rmse': array([0.77225422, 0.7708545 , 0.77160324, 0.76855735, 0.77345334]),
 'test_time': (0.5783567428588867,
  3.9415132999420166,
  0.9490246772766113,
  0.9596729278564453,
  0.5785353183746338)}

# Algorithm

* **SVD**：基于矩阵分解

In [133]:
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8105  0.8049  0.8056  0.8149  0.8091  0.8090  0.0036  
MAE (testset)     0.6539  0.6491  0.6495  0.6583  0.6524  0.6526  0.0034  
Fit time          16.74   16.27   16.10   16.06   18.30   16.69   0.84    
Test time         0.46    0.63    0.63    0.47    0.73    0.59    0.10    


{'fit_time': (16.736965656280518,
  16.266198873519897,
  16.1020667552948,
  16.062939405441284,
  18.297313928604126),
 'test_mae': array([0.65386854, 0.64911034, 0.64945   , 0.65830543, 0.65242825]),
 'test_rmse': array([0.81046074, 0.80494192, 0.8056143 , 0.81488754, 0.80905468]),
 'test_time': (0.4607517719268799,
  0.6332108974456787,
  0.62969970703125,
  0.4707787036895752,
  0.7335309982299805)}

* **KNN**：内存不够!无法运行K近邻

In [8]:
# algo = KNNBasic()
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [9]:
# algo = KNNBaseline()
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

* **NMF**:非负矩阵分解

In [5]:
algo = NMF()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7731  0.7797  0.7829  0.7815  0.7848  0.7804  0.0040  
MAE (testset)     0.5917  0.5960  0.5977  0.5975  0.5986  0.5963  0.0024  
Fit time          18.95   19.04   19.12   18.88   18.98   19.00   0.08    
Test time         0.45    0.44    0.44    0.45    0.44    0.44    0.00    


{'fit_time': (18.951494455337524,
  19.04124689102173,
  19.11853313446045,
  18.884664058685303,
  18.98002815246582),
 'test_mae': array([0.59169409, 0.59604112, 0.59768282, 0.59746873, 0.59855633]),
 'test_rmse': array([0.77310725, 0.7796931 , 0.7829107 , 0.78148127, 0.78475442]),
 'test_time': (0.44800329208374023,
  0.43720126152038574,
  0.4372849464416504,
  0.4477388858795166,
  0.44390130043029785)}

In [11]:
for i in range(10,200,5):
    print("For factors = " + str(i))
    algo = SVD(n_factors=i);
    cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

For factors = 10
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8395  0.8256  0.8385  0.8373  0.8213  0.8324  0.0075  
MAE (testset)     0.6806  0.6724  0.6809  0.6804  0.6654  0.6760  0.0062  
Fit time          5.49    5.53    5.50    5.52    5.49    5.51    0.02    
Test time         0.57    0.49    0.58    0.48    0.48    0.52    0.05    
For factors = 15
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8373  0.8307  0.8296  0.8271  0.8158  0.8281  0.0070  
MAE (testset)     0.6788  0.6753  0.6734  0.6726  0.6602  0.6721  0.0063  
Fit time          6.38    6.03    6.04    6.06    6.05    6.11    0.13    
Test time         0.57    0.48    0.61    0.48    0.48    0.52    0.05    
For factors = 20
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fol

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8045  0.8043  0.8078  0.8109  0.8066  0.8068  0.0024  
MAE (testset)     0.6484  0.6480  0.6525  0.6550  0.6504  0.6509  0.0026  
Fit time          22.66   23.21   22.74   23.43   23.79   23.17   0.42    
Test time         0.64    0.64    0.77    0.74    0.62    0.68    0.06    
For factors = 110
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8064  0.8027  0.8077  0.8051  0.8049  0.8053  0.0017  
MAE (testset)     0.6508  0.6469  0.6517  0.6503  0.6503  0.6500  0.0016  
Fit time          27.21   27.01   26.13   26.56   27.07   26.80   0.40    
Test time         0.79    1.11    0.67    0.68    0.71    0.79    0.16    
For factors = 115
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fo

In [13]:
cross_validate(SVD(n_epochs = 100), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7359  0.7389  0.7346  0.7394  0.7334  0.7364  0.0023  
MAE (testset)     0.5767  0.5826  0.5774  0.5818  0.5773  0.5792  0.0025  
Fit time          106.59  106.43  106.07  106.22  100.58  105.18  2.31    
Test time         0.67    0.66    0.68    0.68    0.75    0.69    0.03    


{'fit_time': (106.59108972549438,
  106.43007731437683,
  106.06523776054382,
  106.21609592437744,
  100.5775990486145),
 'test_mae': array([0.57670509, 0.58264733, 0.57742411, 0.58183619, 0.57730221]),
 'test_rmse': array([0.73591188, 0.73887643, 0.7346145 , 0.73936889, 0.73339469]),
 'test_time': (0.6709907054901123,
  0.6597564220428467,
  0.6842033863067627,
  0.6835579872131348,
  0.7452549934387207)}

## 开始调参

In [ ]:
result = []
for n_factors in range(10,110,10):
    for n_epochs in range(20,120,20):
        for reg_all in [0.001, 0.003, 0.005, 0.007,
                        0.01, 0.03, 0.05, 0.07, 
                        0.1, 0.3, 0.5, 0.7,
                       1, 3, 5, 7 ,
                       10, 15, 20, 30]:
            print("for n_factors = "+str(n_factors) + " n_epochs = "+ str(n_epochs) + " reg_all = "+ str(reg_all))
            algo = SVD(n_factors = n_factors, n_epochs = n_epochs, reg_all = reg_all);
            ret = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True);
            result.append([n_factors,n_epochs,reg_all,np.mean(ret["test_mae"]),np.mean(ret["test_rmse"])])
            pd.DataFrame(result).to_csv("result.csv")

for n_factors = 10 n_epochs = 20 reg_all = 0.001
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7925  0.8102  0.7981  0.8003  0.0074  
MAE (testset)     0.6327  0.6493  0.6384  0.6401  0.0069  
Fit time          7.05    6.67    6.34    6.69    0.29    
Test time         1.39    1.08    1.04    1.17    0.16    
for n_factors = 10 n_epochs = 20 reg_all = 0.003
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8060  0.8161  0.8109  0.8110  0.0041  
MAE (testset)     0.6483  0.6580  0.6527  0.6530  0.0040  
Fit time          6.50    6.43    6.54    6.49    0.05    
Test time         1.23    1.01    1.23    1.15    0.10    
for n_factors = 10 n_epochs = 20 reg_all = 0.005
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8116  0.8073  0.8060  

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7871  0.7835  0.7902  0.7870  0.0027  
MAE (testset)     0.6110  0.6076  0.6147  0.6111  0.0029  
Fit time          8.87    8.86    8.71    8.81    0.07    
Test time         0.90    0.91    0.79    0.87    0.05    
for n_factors = 10 n_epochs = 40 reg_all = 0.005
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7810  0.7752  0.7791  0.7784  0.0024  
MAE (testset)     0.6093  0.6046  0.6060  0.6066  0.0020  
Fit time          8.90    8.71    8.89    8.83    0.09    
Test time         0.80    0.79    0.80    0.80    0.00    
for n_factors = 10 n_epochs = 40 reg_all = 0.007
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7663  0.7736  0.7729  0.7710  0.0033  
MAE (testset)     0.5989  0.6060

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7936  0.7865  0.7927  0.7909  0.0032  
MAE (testset)     0.6152  0.6094  0.6127  0.6125  0.0024  
Fit time          12.78   12.67   12.64   12.70   0.06    
Test time         0.77    0.78    0.76    0.77    0.01    
for n_factors = 10 n_epochs = 60 reg_all = 0.007
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7811  0.7807  0.7830  0.7816  0.0010  
MAE (testset)     0.6067  0.6046  0.6089  0.6067  0.0018  
Fit time          12.74   12.68   12.69   12.70   0.03    
Test time         0.76    0.82    0.88    0.82    0.05    
for n_factors = 10 n_epochs = 60 reg_all = 0.01
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7734  0.7731  0.7685  0.7717  0.0022  
MAE (testset)     0.6036  0.6015 

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7904  0.7927  0.7907  0.7913  0.0010  
MAE (testset)     0.6118  0.6154  0.6125  0.6132  0.0016  
Fit time          19.58   16.81   16.93   17.78   1.28    
Test time         0.78    0.87    0.77    0.80    0.05    
for n_factors = 10 n_epochs = 80 reg_all = 0.01
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7775  0.7742  0.7799  0.7772  0.0023  
MAE (testset)     0.6033  0.6011  0.6045  0.6029  0.0014  
Fit time          16.92   16.87   17.04   16.95   0.07    
Test time         0.76    0.75    0.78    0.76    0.01    
for n_factors = 10 n_epochs = 80 reg_all = 0.03
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7549  0.7487  0.7525  0.7520  0.0025  
MAE (testset)     0.5940  0.5889  

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7884  0.7831  0.7882  0.7866  0.0024  
MAE (testset)     0.6114  0.6062  0.6107  0.6094  0.0023  
Fit time          21.22   21.06   21.07   21.12   0.07    
Test time         0.89    0.76    0.77    0.81    0.06    
for n_factors = 10 n_epochs = 100 reg_all = 0.03
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7471  0.7522  0.7485  0.7493  0.0021  
MAE (testset)     0.5854  0.5891  0.5867  0.5871  0.0015  
Fit time          21.29   21.22   21.03   21.18   0.11    
Test time         0.77    0.87    0.77    0.80    0.05    
for n_factors = 10 n_epochs = 100 reg_all = 0.05
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7501  0.7498  0.7459  0.7486  0.0019  
MAE (testset)     0.5940  0.5936

...

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9169  0.9145  0.9138  0.9151  0.0013  
MAE (testset)     0.7543  0.7514  0.7513  0.7524  0.0014  
Fit time          18.11   17.87   17.92   17.97   0.10    
Test time         0.78    0.88    0.77    0.81    0.05    
for n_factors = 10 n_epochs = 60 reg_all = 3
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9639  0.9562  0.9621  0.9607  0.0033  
MAE (testset)     0.7957  0.7896  0.7932  0.7928  0.0025  
Fit time          18.21   17.92   18.20   18.11   0.14    
Test time         0.78    0.78    0.77    0.78    0.00    
for n_factors = 10 n_epochs = 60 reg_all = 5
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9725  0.9778  0.9738  0.9747  0.0023  
MAE (testset)     0.8027  0.8072  0.8055

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9583  0.9632  0.9616  0.9610  0.0020  
MAE (testset)     0.7912  0.7951  0.7931  0.7931  0.0016  
Fit time          23.89   23.92   23.75   23.85   0.08    
Test time         0.77    0.76    0.77    0.77    0.00    
for n_factors = 10 n_epochs = 80 reg_all = 5
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9756  0.9748  0.9713  0.9739  0.0019  
MAE (testset)     0.8066  0.8046  0.8028  0.8046  0.0016  
Fit time          23.99   23.81   23.81   23.87   0.09    
Test time         0.78    0.92    0.78    0.82    0.07    
for n_factors = 10 n_epochs = 80 reg_all = 7
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9827  0.9804  0.9810  0.9814  0.0010  
MAE (testset)     0.8121  0.8101  0.8112

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9738  0.9738  0.9794  0.9757  0.0026  
MAE (testset)     0.8057  0.8035  0.8086  0.8059  0.0021  
Fit time          30.01   29.79   29.86   29.89   0.09    
Test time         0.89    0.77    0.77    0.81    0.06    
for n_factors = 10 n_epochs = 100 reg_all = 7
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9828  0.9794  0.9807  0.9810  0.0014  
MAE (testset)     0.8118  0.8091  0.8115  0.8108  0.0012  
Fit time          30.28   29.64   31.07   30.33   0.58    
Test time         0.78    0.79    0.77    0.78    0.01    
for n_factors = 10 n_epochs = 100 reg_all = 10
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9845  0.9904  0.9857  0.9869  0.0026  
MAE (testset)     0.8142  0.8184  0.8

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9817  0.9816  0.9800  0.9811  0.0008  
MAE (testset)     0.8106  0.8117  0.8099  0.8107  0.0007  
Fit time          7.09    6.98    7.10    7.06    0.06    
Test time         0.77    0.78    0.79    0.78    0.01    
for n_factors = 10 n_epochs = 20 reg_all = 10
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9845  0.9852  0.9842  0.9846  0.0004  
MAE (testset)     0.8138  0.8150  0.8138  0.8142  0.0006  
Fit time          7.00    6.95    7.01    6.99    0.03    
Test time         0.78    0.88    0.77    0.81    0.05    
for n_factors = 10 n_epochs = 20 reg_all = 15
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9921  0.9914  0.9878  0.9904  0.0019  
MAE (testset)     0.8208  0.8190  0.81

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9856  0.9864  0.9860  0.9860  0.0003  
MAE (testset)     0.8147  0.8160  0.8148  0.8152  0.0006  
Fit time          14.26   13.74   13.94   13.98   0.21    
Test time         0.87    0.76    0.76    0.80    0.05    
for n_factors = 10 n_epochs = 40 reg_all = 15
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9895  0.9933  0.9873  0.9900  0.0025  
MAE (testset)     0.8184  0.8209  0.8164  0.8186  0.0018  
Fit time          14.52   15.33   13.79   14.55   0.63    
Test time         1.07    0.78    0.79    0.88    0.13    
for n_factors = 10 n_epochs = 40 reg_all = 20
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9912  0.9919  0.9931  0.9921  0.0008  
MAE (testset)     0.8199  0.8202  0.82

In [8]:
result2 = []
for n_factors in range(20,60,10):
    for n_epochs in range(100,320,70):
        for reg_all in [0.02, 0.03, 0.05, 0.07]:
            print("for n_factors = "+str(n_factors) + " n_epochs = "+ str(n_epochs) + " reg_all = "+ str(reg_all))
            algo = SVD(n_factors = n_factors, n_epochs = n_epochs, reg_all = reg_all);
            ret = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True);
            result2.append([n_factors,n_epochs,reg_all,np.mean(ret["test_mae"]),np.mean(ret["test_rmse"])])
            pd.DataFrame(result2).to_csv("result2.csv")

for n_factors = 20 n_epochs = 100 reg_all = 0.02
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7584  0.7603  0.7589  0.7592  0.0008  
MAE (testset)     0.5924  0.5945  0.5936  0.5935  0.0009  
Fit time          23.62   23.71   23.53   23.62   0.07    
Test time         0.79    0.72    0.79    0.77    0.03    
for n_factors = 20 n_epochs = 100 reg_all = 0.03
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7465  0.7469  0.7475  0.7470  0.0004  
MAE (testset)     0.5853  0.5858  0.5869  0.5860  0.0007  
Fit time          23.72   23.38   23.42   23.51   0.15    
Test time         0.81    0.79    0.79    0.79    0.01    
for n_factors = 20 n_epochs = 100 reg_all = 0.05
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7472  0.7435  0.7462  

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7472  0.7446  0.7456  0.7458  0.0011  
MAE (testset)     0.5848  0.5825  0.5835  0.5836  0.0009  
Fit time          49.95   54.17   52.63   52.25   1.75    
Test time         0.86    0.84    0.74    0.81    0.05    
for n_factors = 30 n_epochs = 170 reg_all = 0.05
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7377  0.7377  0.7399  0.7384  0.0010  
MAE (testset)     0.5816  0.5805  0.5814  0.5812  0.0005  
Fit time          51.18   51.07   51.89   51.38   0.37    
Test time         0.87    1.18    1.00    1.01    0.13    
for n_factors = 30 n_epochs = 170 reg_all = 0.07
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7386  0.7371  0.7384  0.7380  0.0007  
MAE (testset)     0.5838  0.5830

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7367  0.7296  0.7335  0.7333  0.0029  
MAE (testset)     0.5782  0.5734  0.5759  0.5758  0.0019  
Fit time          72.56   72.06   73.52   72.71   0.61    
Test time         0.77    0.73    0.77    0.76    0.01    
for n_factors = 40 n_epochs = 240 reg_all = 0.07
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7331  0.7340  0.7360  0.7344  0.0012  
MAE (testset)     0.5776  0.5793  0.5808  0.5792  0.0013  
Fit time          78.54   80.81   76.89   78.75   1.61    
Test time         1.13    0.79    0.82    0.91    0.15    
for n_factors = 40 n_epochs = 310 reg_all = 0.02
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7495  0.7493  0.7493  0.7494  0.0001  
MAE (testset)     0.5873  0.5860

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7347  0.7328  0.7323  0.7333  0.0010  
MAE (testset)     0.5795  0.5783  0.5786  0.5788  0.0005  
Fit time          127.00  123.52  124.05  124.86  1.53    
Test time         0.92    0.91    0.91    0.91    0.00    


In [10]:
result3 = []
for n_factors in range(50,80,10):
    for n_epochs in range(200,420,50):
        for reg_all in [0.03, 0.04, 0.05, 0.06, 0.07]:
            print("for n_factors = "+str(n_factors) + " n_epochs = "+ str(n_epochs) + " reg_all = "+ str(reg_all))
            algo = SVD(n_factors = n_factors, n_epochs = n_epochs, reg_all = reg_all);
            ret = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True);
            result3.append([n_factors,n_epochs,reg_all,np.mean(ret["test_mae"]),np.mean(ret["test_rmse"])])
            pd.DataFrame(result3).to_csv("result3.csv")

for n_factors = 50 n_epochs = 200 reg_all = 0.03
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7378  0.7345  0.7380  0.7368  0.0016  
MAE (testset)     0.5783  0.5755  0.5778  0.5772  0.0012  
Fit time          77.93   79.47   82.52   79.97   1.91    
Test time         0.87    0.90    1.39    1.05    0.24    
for n_factors = 50 n_epochs = 200 reg_all = 0.04
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7340  0.7316  0.7355  0.7337  0.0016  
MAE (testset)     0.5764  0.5739  0.5781  0.5762  0.0017  
Fit time          88.07   78.17   77.20   81.15   4.91    
Test time         0.89    0.87    0.89    0.88    0.01    
for n_factors = 50 n_epochs = 200 reg_all = 0.05
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7353  0.7350  0.7304  

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7306  0.7349  0.7295  0.7317  0.0024  
MAE (testset)     0.5739  0.5761  0.5727  0.5742  0.0014  
Fit time          166.58  148.29  147.82  154.23  8.73    
Test time         0.86    0.82    0.81    0.83    0.02    
for n_factors = 50 n_epochs = 400 reg_all = 0.05
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7308  0.7300  0.7323  0.7310  0.0010  
MAE (testset)     0.5746  0.5737  0.5751  0.5745  0.0006  
Fit time          147.34  147.46  144.66  146.49  1.29    
Test time         0.83    1.03    0.80    0.89    0.10    
for n_factors = 50 n_epochs = 400 reg_all = 0.06
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7301  0.7317  0.7343  0.7320  0.0018  
MAE (testset)     0.5745  0.5764

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7281  0.7305  0.7300  0.7295  0.0010  
MAE (testset)     0.5727  0.5741  0.5734  0.5734  0.0006  
Fit time          164.83  166.00  166.33  165.72  0.65    
Test time         0.91    1.38    0.75    1.02    0.27    
for n_factors = 60 n_epochs = 350 reg_all = 0.06
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7312  0.7272  0.7345  0.7310  0.0030  
MAE (testset)     0.5762  0.5727  0.5780  0.5756  0.0022  
Fit time          157.71  155.56  171.15  161.47  6.90    
Test time         1.15    0.88    0.90    0.98    0.12    
for n_factors = 60 n_epochs = 350 reg_all = 0.07
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7310  0.7343  0.7311  0.7321  0.0015  
MAE (testset)     0.5764  0.5790

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7272  0.7319  0.7322  0.7305  0.0023  
MAE (testset)     0.5726  0.5771  0.5765  0.5754  0.0020  
Fit time          137.93  141.80  142.35  140.69  1.97    
Test time         0.85    0.86    0.85    0.85    0.00    
for n_factors = 70 n_epochs = 300 reg_all = 0.07
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7290  0.7325  0.7329  0.7315  0.0017  
MAE (testset)     0.5754  0.5793  0.5782  0.5776  0.0017  
Fit time          142.65  134.06  138.53  138.41  3.50    
Test time         0.81    0.80    0.87    0.83    0.03    
for n_factors = 70 n_epochs = 350 reg_all = 0.03
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7272  0.7316  0.7301  0.7296  0.0018  
MAE (testset)     0.5701  0.5733

## 调参结束

In [11]:
algo = SVD(n_factors = 70, n_epochs = 350, reg_all = 0.04);
ret = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True);

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7287  0.7273  0.7247  0.7269  0.0017  
MAE (testset)     0.5733  0.5707  0.5699  0.5713  0.0014  
Fit time          541.33  513.09  851.06  635.16  153.10  
Test time         1.51    1.36    0.92    1.27    0.25    
